# Importation des packages

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import pickle
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import matplotlib.pyplot as plt

import cv2
import math
from glob import glob

# Syncrhonisation avec Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


# Variables d'environnement

In [3]:
PATH_DATA = 'gdrive/MyDrive/AIForYou/Datasets/supervisée/classification/non_structurée/Tetris/first_line/'

# Importation des données

# Chargement des données sauvegardées

In [28]:
training_data = pd.read_csv(f'{PATH_DATA}train_data-first_line.csv')
training_data.shape

(84516, 4)

In [29]:
testing_data = pd.read_csv(f'{PATH_DATA}test_data-first_line.csv')
testing_data.shape

(1119, 4)

In [30]:
training_data_images = np.load(f'{PATH_DATA}train_data_images-first_line.npy')
training_data_images.shape

(84516, 20, 10, 1)

In [31]:
testing_data_images = np.load(f'{PATH_DATA}test_data_images-first_line.npy')
testing_data_images.shape

(1119, 20, 10, 1)

# Mettre les labels au bon format

In [32]:
def output_form(data):
    column = data.pop('column')
    column = list(map(lambda x : int(x), column))
    column = to_categorical(column)
    column = np.array(column)
    rotation = data.pop('rotation')
    rotation = list(map(lambda x : int(x), rotation))
    rotation = to_categorical(rotation)
    rotation = np.array(rotation)
    return (rotation, column)

In [33]:
y_train = output_form(training_data.loc[:, ['column', 'rotation']])

In [34]:
new_train = np.zeros([training_data.shape[0], 40])
cmpt = 0
for rot, col in zip(y_train[0], y_train[1]):

  new_train[cmpt, :] = np.dot(rot.T.reshape(-1, 1), col.reshape(1, -1)).flatten()
  cmpt += 1

In [35]:
np.sum(new_train, axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

In [36]:
y_test = output_form(testing_data.loc[:, ['column', 'rotation']])

In [37]:
new_test = np.zeros([testing_data.shape[0], 40])
cmpt = 0
for rot, col in zip(y_test[0], y_test[1]):

  new_test[cmpt, :] = np.dot(rot.T.reshape(-1, 1), col.reshape(1, -1)).flatten()
  cmpt += 1

In [38]:
np.sum(new_test, axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

# Distribution des données

In [39]:
np.sum(new_train, axis=0)

array([10389.,  5145.,  5568.,  6167.,  6111.,  5669.,  6252.,  7004.,
        2412.,     0.,  5850.,   688.,  1241.,  1517.,  1339.,  1456.,
        1458.,   858.,  2440.,  1478.,  1155.,   445.,   731.,   797.,
         733.,   677.,   404.,  1138.,     0.,     0.,   600.,   182.,
         398.,   331.,   343.,   391.,   317.,   151.,  2681.,     0.])

# Création des générateurs

In [40]:
class image_generator():

    # Class is a dataset wrapper for better training performance
    def __init__(self, image_list, target_list, norm=False, batch_size=8, shuffle=True, data_aug=True, no_whole=False):
        self.image_list = image_list
        self.target_list = target_list
        self.id_list = np.arange(image_list.shape[0])
        self.batch_size = batch_size
        self.len = math.ceil(len(self.image_list) / self.batch_size)
        self.idx = 0
        self.shuffle = shuffle
        self.norm = norm
        self.no_whole = no_whole
        self.data_aug = data_aug
        if self.shuffle :
            np.random.shuffle(self.id_list)

    def __norm__(self, batch):
        return np.array([ (x - np.min(x) )/ (np.max(x) - np.min(x)) for x in batch])

    def __getitem__(self):
        if (self.idx + 1) * self.batch_size > len(self.image_list) :
            idx_end = len(self.image_list)
        else :
            idx_end = (self.idx + 1) * self.batch_size


        id_list_batch = self.id_list[self.idx * self.batch_size:idx_end]
        batch_img = self.image_list[id_list_batch]

        if self.norm :
            batch_img = self.__norm__(batch_img)

        if self.data_aug:
            batch_img = np.array([self.data_augmentation(x)for x in batch_img])

        if self.no_whole:
            batch_img = np.array([self.no_whole(x)for x in batch_img])

        return batch_img

    def __gettarget__(self):
        if (self.idx + 1) * self.batch_size > len(self.image_list) :
            idx_end = len(self.image_list)
        else :
            idx_end = (self.idx + 1) * self.batch_size

        id_list_batch = self.id_list[self.idx * self.batch_size:idx_end]

        target_batch = self.target_list[id_list_batch]


        return target_batch



    def __getbatch__(self):
        batch = self.__getitem__()

        x = np.array(batch)
        y = self.__gettarget__()
        return x, y

    # Generate flow of data
    def loader(self):
        # load data from somwhere with Python, and yield them
        while True:
            batch_input, batch_output = self.__getbatch__()
            self.__iter__()
            yield (batch_input, batch_output)

    def __iter__(self):
        if self.idx + 2 > self.len :
            self.idx = 0
            if self.shuffle :
                np.random.shuffle(self.id_list)
        else :
            self.idx += 1

    def get_len(self):
        return self.len

    def get_minimum_maximum_height(self, game_matrix):
        lines, columns, _ = game_matrix.shape
        minimum_height = np.inf
        maximum_height = -np.inf
        for column in np.arange(columns):
          for line in np.arange(2,lines):
            if game_matrix[line, column, 0] == 1 :

              if lines-line < minimum_height:
                minimum_height = lines-line
              if lines-line > maximum_height:
                maximum_height = lines-line
              break

        if minimum_height == np.inf or maximum_height==-np.inf:
          maximum_height = 0
          minimum_height = 0
        return maximum_height, minimum_height

    def increase_row(self, game_matrix, maximum_height):

      number_row = np.random.randint(18-maximum_height+1)

      new_game_matrix = np.zeros(game_matrix.shape)

      new_game_matrix[:2, :, :] = game_matrix[:2, :, :]

      new_game_matrix[20-maximum_height-number_row:20-number_row, :, :] = game_matrix[20-maximum_height:, :, :]

      new_game_matrix[20-number_row:, :, :] = 1

      return new_game_matrix

    def reduce_row(self, game_matrix, minimum_height, maximum_height):

      number_row = np.random.randint(minimum_height+1)

      new_game_matrix = np.zeros(game_matrix.shape)

      new_game_matrix[:2, :, :] = game_matrix[:2, :, :]

      new_game_matrix[-((maximum_height-number_row)):, :, :] = game_matrix[20-maximum_height:20-number_row, :, :]

      return new_game_matrix


    def data_augmentation(self, game_matrix):

      maximum_height, minimum_height = self.get_minimum_maximum_height(game_matrix)


      add_row = np.random.randint(2)

      if add_row and maximum_height<18 :
        game_matrix = self.increase_row(game_matrix, maximum_height)


      elif not(add_row) and minimum_height>0 and maximum_height>minimum_height:

        game_matrix = self.reduce_row(game_matrix, minimum_height, maximum_height)
        maximum_height, minimum_height = self.get_minimum_maximum_height(game_matrix)


      return game_matrix

    def no_whole(self, game_matrix):

      gamme_matrix_without_current_tetromino = game_matrix.copy()

      width, colmun, _ = gamme_matrix_without_current_tetromino.shape

      for col in np.arange(colmun):
        first_tetromino = np.argmax(gamme_matrix_without_current_tetromino[2:, col, :])

        gamme_matrix_without_current_tetromino[first_tetromino:, col, :]=1

      return gamme_matrix_without_current_tetromino



In [42]:
train_datagen = image_generator(training_data_images, new_train, batch_size=512, norm=False, data_aug=True, no_whole=False, shuffle=True)
test_datagen = image_generator(testing_data_images, new_test, batch_size=512, norm=False, data_aug=False, no_whole=False, shuffle=True)

In [43]:
for i in range(5):
  x, y = next(train_datagen.loader())
  print(x[0, :, :, 0])
  print(x.shape)
  print(y.shape)

[[0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
(512, 20, 10, 1)
(512, 40)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0

# Initialisation du modèle

In [19]:
def base_model(inputs):
  x = tf.keras.layers.ZeroPadding2D(padding=(2, 2))(inputs)
  x = tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation='relu')(x)
  x = tf.keras.layers.AveragePooling2D((2, 2))(x)
  #x = tf.keras.layers.Dropout(0.1)(x)
  x = tf.keras.layers.Conv2D(16, (3, 3), padding="same", activation='relu')(x)
  x = tf.keras.layers.AveragePooling2D((2, 2))(x)
  #x = tf.keras.layers.Dropout(0.1)(x)
  x = tf.keras.layers.Conv2D(32, (3, 3), padding="same", activation='relu')(x)
  x = tf.keras.layers.AveragePooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation='relu')(x)
  x = tf.keras.layers.Flatten()(x)
  #x = tf.keras.layers.Dense(64, activation='relu')(x)
  #x = tf.keras.layers.Dropout(0.1)(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  #x = tf.keras.layers.Dropout(0.1)(x)
  output = tf.keras.layers.Dense(units = '40', activation = 'softmax')(x)
  model = tf.keras.models.Model(inputs=inputs, outputs = output)

  return model

In [20]:
inputs = tf.keras.layers.Input(shape=(20, 10, 1))

In [21]:
model = base_model(inputs)

In [22]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss = tf.keras.losses.CategoricalCrossentropy(),
             metrics=tf.keras.metrics.AUC())

In [23]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 10, 1)]       0         
                                                                 
 zero_padding2d (ZeroPaddin  (None, 24, 14, 1)         0         
 g2D)                                                            
                                                                 
 conv2d (Conv2D)             (None, 24, 14, 8)         80        
                                                                 
 average_pooling2d (Average  (None, 12, 7, 8)          0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 7, 16)         1168      
                                                                 
 average_pooling2d_1 (Avera  (None, 6, 3, 16)          0     

# Entraînement du moèle

In [24]:
PATH_model = 'gdrive/MyDrive/AIForYou/applications/tetris/training_model/checkpoint_epoch-{epoch}_val_loss-{val_loss:.4f}.hdf5'

In [25]:
callback = tf.keras.callbacks.ModelCheckpoint(PATH_model, monitor="val_loss", mode="min", save_best_only=True)

In [26]:
model.load_weights('gdrive/MyDrive/AIForYou/applications/tetris/training_model/checkpoint_epoch-209_val_loss-1.4781.hdf5')

In [ ]:
history = model.fit(train_datagen.loader(),
                    batch_size=train_datagen.batch_size,
                    steps_per_epoch= train_datagen.get_len(),
                    initial_epoch=227,
                    epochs=10000,
                    validation_data=test_datagen.loader(),
                    validation_steps=test_datagen.get_len(),
                    validation_batch_size=test_datagen.batch_size,
                    callbacks=[callback])

Epoch 228/10000
166/166 [==============================] - 17s 102ms/step - loss: 1.5920 - auc: 0.9603 - val_loss: 1.5202 - val_auc: 0.9637
Epoch 229/10000
166/166 [==============================] - 16s 94ms/step - loss: 1.5893 - auc: 0.9602 - val_loss: 1.4729 - val_auc: 0.9645
Epoch 230/10000
  1/166 [..............................] - ETA: 3s - loss: 1.6778 - auc: 0.9540

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


166/166 [==============================] - 17s 100ms/step - loss: 1.5930 - auc: 0.9602 - val_loss: 1.4870 - val_auc: 0.9649
Epoch 231/10000
166/166 [==============================] - 15s 93ms/step - loss: 1.5947 - auc: 0.9602 - val_loss: 1.4752 - val_auc: 0.9648
Epoch 232/10000
166/166 [==============================] - 15s 91ms/step - loss: 1.5917 - auc: 0.9603 - val_loss: 1.4860 - val_auc: 0.9651
Epoch 233/10000
166/166 [==============================] - 15s 91ms/step - loss: 1.5931 - auc: 0.9601 - val_loss: 1.4904 - val_auc: 0.9657
Epoch 234/10000
166/166 [==============================] - 15s 91ms/step - loss: 1.5856 - auc: 0.9605 - val_loss: 1.4751 - val_auc: 0.9655
Epoch 235/10000
166/166 [==============================] - 15s 92ms/step - loss: 1.5907 - auc: 0.9603 - val_loss: 1.4891 - val_auc: 0.9640
Epoch 236/10000
166/166 [==============================] - 15s 92ms/step - loss: 1.5914 - auc: 0.9603 - val_loss: 1.5262 - val_auc: 0.9630
Epoch 237/10000
166/166 [=================

In [ ]:
#model.save_weights('gdrive/MyDrive/AIForYou/applications/tetris/training_model/checkpoint_epoch-4533_val_loss-1.3344.hdf5')

In [ ]:
matrix = np.array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [ ]:
matrix.shape

In [ ]:
x, y = next(train_datagen.loader())

In [ ]:
x[1, :, :, 0]

In [ ]:
y

In [ ]:
model.predict()